# Merge Dask DataFrames

## Merge Large Dask DataFrame to Small Pandas DataFrame

In [1]:
import coiled

In [2]:
import pandas as pd
import dask.dataframe as dd
import dask

In [4]:
cluster = coiled.Cluster(
    name="dask-merge",
    n_workers=50,
    backend_options={'spot':'True'},
    software='coiled-examples/numpy-zarr',
)

Output()

Found software environment build
Created fw rule: inbound [8786-8787] [0.0.0.0/0] []
Created FW rules: coiled-dask-rrpelgr71-95596-firewall
Created fw rule: cluster [0-65535] [None] [coiled-dask-rrpelgr71-95596-firewall -> coiled-dask-rrpelgr71-95596-firewall]
Created FW rules: coiled-dask-rrpelgr71-95596-cluster-firewall
Created fw rule: cluster [0-65535] [None] [coiled-dask-rrpelgr71-95596-cluster-firewall -> coiled-dask-rrpelgr71-95596-cluster-firewall]
Created scheduler VM: coiled-dask-rrpelgr71-95596-scheduler (type: t3a.medium, ip: ['44.200.57.30'])


In [5]:
from distributed import Client
client = Client(cluster)
client

/Users/rpelgrim/mambaforge/envs/coiled-base/lib/python3.9/site-packages/distributed/client.py:1131: VersionMismatchWarning: Mismatched versions found

+-------------+---------------+---------------+---------------+
| Package     | client        | scheduler     | workers       |
+-------------+---------------+---------------+---------------+
| dask        | 2021.11.2     | 2021.11.1     | 2021.11.1     |
| distributed | 2021.11.2     | 2021.11.1     | 2021.11.1     |
| lz4         | 3.1.10        | 3.1.3         | 3.1.3         |
| msgpack     | 1.0.3         | 1.0.2         | 1.0.2         |
| numpy       | 1.21.4        | 1.21.2        | 1.21.2        |
| pandas      | 1.3.5         | 1.3.4         | 1.3.4         |
| python      | 3.9.7.final.0 | 3.9.6.final.0 | 3.9.6.final.0 |
+-------------+---------------+---------------+---------------+
Notes: 
-  msgpack: Variation is ok, as long as everything is above 0.6
  warnings.warn(version_module.VersionMismatchWarning(msg[0]["warning"]))

<Client: 'tls://10.4.3.166:8786' processes=31 threads=62, memory=233.14 GiB>

In [12]:
large = dask.datasets.timeseries(freq="10s", npartitions=10)
small = dask.datasets.timeseries(freq="1D", dtypes={"z": int})

In [13]:
large.dtypes

id        int64
name     object
x       float64
y       float64
dtype: object

In [14]:
large.to_parquet(
    's3://coiled-datasets/dask-merge/large.parquet',
    engine='pyarrow',
)

CancelledError: ('metadata-to-parquet-136e9b0648e984691f0dfb0bb5273560', 0)

In [4]:
small = small.compute()

In [5]:
small

,z
timestamp,
2000-01-01,984
2000-01-02,1021
2000-01-03,979
2000-01-04,1050
2000-01-05,1003
2000-01-06,1009
2000-01-07,984
2000-01-08,979
2000-01-09,977


In [6]:
result = large.merge(small, how="left", on=["timestamp"])

In [7]:
result.head()

,id,name,x,y,z
timestamp,,,,,
2000-01-01 00:00:00,1021,Laura,-0.198011,-0.835659,984.0
2000-01-01 00:00:10,993,George,-0.711381,0.904073,NaN
2000-01-01 00:00:20,915,Laura,0.807961,-0.473370,NaN
2000-01-01 00:00:30,953,Ray,0.120000,0.743861,NaN
2000-01-01 00:00:40,979,Tim,-0.104925,0.134858,NaN


## Merge two Dask DataFrames

In [8]:
left = dask.datasets.timeseries(dtypes={"foo": int})
right_one = dask.datasets.timeseries(dtypes={"bar": int})
right_two = dask.datasets.timeseries(dtypes={"baz": int})

In [9]:
# timeseries returns a dataframe indexed by
# timestamp, we don't need to set_index.

# left.set_index("timestamp")

#left.to_parquet("left", overwrite=True)
#left = dd.read_parquet("left")

# If the dataframe can fit in RAM, you can also use persist

# left = left.persist()


result1 = left.merge(
    right_one, how="left", left_index=True, right_index=True)

In [13]:
result2 = result1.merge(
    right_two, how="left", left_index=True, right_index=True)

In [11]:
left.head()

,foo
timestamp,
2000-01-01 00:00:00,1032
2000-01-01 00:00:01,993
2000-01-01 00:00:02,993
2000-01-01 00:00:03,995
2000-01-01 00:00:04,995


In [10]:
result1.head()

,foo,bar
timestamp,,
2000-01-01 00:00:00,1032,1016
2000-01-01 00:00:01,993,949
2000-01-01 00:00:02,993,1004
2000-01-01 00:00:03,995,1023
2000-01-01 00:00:04,995,1042


In [14]:
result2.head()

,foo,bar,baz
timestamp,,,
2000-01-01 00:00:00,1032,1016,1014
2000-01-01 00:00:01,993,949,1010
2000-01-01 00:00:02,993,1004,984
2000-01-01 00:00:03,995,1023,1005
2000-01-01 00:00:04,995,1042,981
